# ФОРМУЛИРОВКА ЗАДАЧИ

Используем файл data_no_header.txt

Итак, мы готовы перейти к решению основной задачи этого модуля - подсчету ROI. Сформулируем задачу.

Для статистики покупок в файле data.txt необходимо посчитать какую прибыль нам принес каждый заказ и каждый канал. Имеется набор условий по подсчету расходов:

С партнерами канала cpa-partners мы делимся долей выручки. Т. е. к расходам в столбце cost надо прибавлять определенный процент от столбца amount_paid. Каждый канал cpa-partners имеет свой размер комиссии:

    - burgerclub 30%

    - food-delivery 25%

У канала cpc-partners схема попроще - фиксированная стоимость с каждого заказа:

    - city-magazine 7 рублей

    - foody 9 рублей

У остальных каналов считаем расходы за период фиксированными. Условно 1000 рублей на все заказы. Т. е. надо равномерно распределить эти расходы на каждый заказ на все каналы, кроме cpa-partners или cpc-partners.

В итоге для каждого типа канала (назовем их CPA, CPC и Fixed) необходимо посчитать показатель ROI (return on investment). Т. е. отношение прибыли (amount_paid - cost) к расходам cost

## ПОДГОТОВКА ДАННЫХ
Заведем словари, в которых будут содержаться комиссии трех типов

In [1]:
cpa_commission = {
    'burgerclub': 0.3,
    'food-delivery': 0.25
}

cpc_commission = {
    'city-magazine': 7,
    'foody': 9
}

В файле 266 заказов из остальных каналов, поэтому можно рассчитать фиксированную стоимость для этого случая как 1000 / 266. Примем значение равным 4 рублям.

In [2]:
fixed_commission = 4

## УПРАЖНЕНИЕ "НА ПОДУМАТЬ"
В подобных задачах прежде чем писать код лучше заранее набросать небольшой план. Например, написать основные шаги, по которым вы будете строить алгоритм. Т. к. в условии прописан набор правил, то надо учитывать, что в будущем эти правила могут измениться или пополниться новыми. Скорее всего для обработки правил лучше написать отдельную функцию, чтобы при любых изменениях ее можно было быстро проверить.

Какие еще шаги алгоритма вы можете предусмотреть, исходя из собственного опыта?

## ОПРЕДЕЛЕНИЕ КОМИССИИ ПО ИСТОЧНИКУ
В предыдущем упражнении вы продумали первые шаги (если нет, то крайне советуем это сделать перед прочтением нашего решения задачи). Теперь обратите внимание, какие шаги предлагаем мы. 

Файл data_no_header.txt

Напишем функцию, которая по названию источника source и сумме заказа будет возвращать значение комиссии.
```python

In [3]:
def costs_classification( amount_paid, source ):
    """
    Функция по названию канала source возвращает размер комиссии.
    Размеры комиссий берутся из словарей cpa_commission, cpc_commission и значения fixed_commission
    """
    # если источник source входит в словарь cpa_commission, то возвращаем долю от выручки
    if source in cpa_commission:
        return amount_paid * cpa_commission[ source ]
    # в случае партнеров CPC стоимость расходов постоянная
    if source in cpc_commission:
        return cpc_commission[ source ]
    # если ни один случай не подходит, то возвращаем фиксированную стоимость fixed_commission
    return fixed_commission

Протестируем как работает. В качестве выручки возьмем 10 рублей

In [4]:
costs_classification( 10, 'burgerclub' ) # 30% от 10 рублей

3.0

Результат:
```python
3.0
```

In [5]:
costs_classification( 10, 'foody' ) # фиксированная сумма в 9 рублей

9

Результат:
```python
9
```

## ПОДСЧЕТ РАСХОДА ДЛЯ СТРОКИ
Похоже все верно. Теперь можем построить функцию, которая для данной строчки line считает расходы. Т. к. нам нужен показатель ROI, то давайте на выходе функция будет возвращать еще и доход. Чтобы по этим двум значениям при проходе в цикле сразу считать ROI.

Для расчета суммарного расхода необходимо взять сумму из столбца cost и прибавить комиссию, которая вычисляет функция costs_classification. Наша строка имеет вид:

In [6]:
line = ['40443', '05.10.2016 23:18', '1010', '0,000925926', 'seo', 'google', '0', '6243', '20,20']

Сначала выделим из строки название источника, расход (столбец cost) и выручку (столбец amount_paid)

##### Функция для выбора нужных полей

In [7]:
def expense_and_income( line ):
    """
    Функция для строки line возвращает итоговый расход и доход
    """
    source = line[5]
    amount_paid = float(line[-1].replace(',', '.'))
    cost = float(line[6].replace(',', '.'))
    return source, cost, amount_paid

Проверим корректность работы

In [8]:
expense_and_income( line )

('google', 0.0, 20.2)

Теперь добавим к расчету комиссию партнера. Используем для этого уже проверенную функцию costs_classification

In [9]:
def expense_and_income( line ):
    """
    Функция для строки line возвращает итоговый расход и доход
    """
    source = line[5]
    amount_paid = float(line[-1].replace(',', '.'))
    cost = float(line[6].replace(',', '.'))
    partner_comission = costs_classification(amount_paid, source)
    return source, cost, amount_paid, partner_comission

In [10]:
expense_and_income(line)

('google', 0.0, 20.2, 4)

## СЧИТАЕМ ROI
Осталось прибавить сумму partner_comission к расходам cost и потестировать корректность работы нашей функции

In [11]:
def expense_and_income( line ):
    """
    Функция для строки line возвращает итоговый расход и доход
    """
    source = line[5]
    amount_paid = float( line[-1].replace( ',', '.' ) )
    cost = float( line[6].replace( ',', '.' ) )
    partner_comission = costs_classification( amount_paid, source )
    return source, cost + partner_comission, amount_paid

Можно провести тесты

должны получить расходы 20.2 * 0.3 + 1 = 7.06

In [12]:
expense_and_income( ['40443', '05.10.2016 23:18', '1010', '0,000925926', 'seo', 'burgerclub', '1', '6243', '20,20'] )

('burgerclub', 7.06, 20.2)

Теперь прогоним через функцию весь файл и запишем результат в словарь roi_stats

In [13]:
roi_stats = {}

with open('./module_2_files/data_no_header.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t')
        source, cost, income = expense_and_income( line )
        if source in roi_stats:
            roi_stats[source]['cost'] += cost
            roi_stats[source]['income'] += income
        else:
            roi_stats[source] = {}
            roi_stats[source]['cost'] = cost
            roi_stats[source]['income'] = income

In [14]:
roi_stats

{'google': {'cost': 314.9, 'income': 1318.8000000000002},
 'yandex': {'cost': 423.36999999999983, 'income': 1818.1999999999998},
 'promo': {'cost': 272.0, 'income': 1180.8},
 'burgerclub': {'cost': 70.276, 'income': 185.39999999999998},
 'food-delivery': {'cost': 67.68, 'income': 173.40000000000003},
 'city-magazine': {'cost': 35.85, 'income': 81.0},
 'foody': {'cost': 27.299999999999997, 'income': 66.0},
 'newsletter': {'cost': 20.0, 'income': 98.20000000000002},
 'facebook': {'cost': 20.0, 'income': 90.60000000000001},
 'direct': {'cost': 20.0, 'income': 88.4},
 'vk': {'cost': 8.0, 'income': 29.0}}

# ДОБАВЛЯЕМ РАСЧЕТ ПО ФОРМУЛЕ
Для каждого источника в словаре roi_stats есть значение расходов (cost) и дохода (income). Заведем новую ячейку и добавим еще один ключ с расчетом ROI:

In [15]:
for source, data in roi_stats.items():
    data['roi'] = (data['income'] - data['cost']) / data['cost']

Итоговый результат:

In [19]:
for source, data in roi_stats.items():
    print('{}\t\t{:.2f}'.format(source, data['roi']))

google		3.19
yandex		3.29
promo		3.34
burgerclub		1.64
food-delivery		1.56
city-magazine		1.26
foody		1.42
newsletter		3.91
facebook		3.53
direct		3.42
vk		2.62


In [20]:
roi_stats

{'google': {'cost': 314.9,
  'income': 1318.8000000000002,
  'roi': 3.1879961892664346},
 'yandex': {'cost': 423.36999999999983,
  'income': 1818.1999999999998,
  'roi': 3.2945886576753205},
 'promo': {'cost': 272.0, 'income': 1180.8, 'roi': 3.341176470588235},
 'burgerclub': {'cost': 70.276,
  'income': 185.39999999999998,
  'roi': 1.6381695031020547},
 'food-delivery': {'cost': 67.68,
  'income': 173.40000000000003,
  'roi': 1.5620567375886527},
 'city-magazine': {'cost': 35.85, 'income': 81.0, 'roi': 1.2594142259414225},
 'foody': {'cost': 27.299999999999997,
  'income': 66.0,
  'roi': 1.4175824175824179},
 'newsletter': {'cost': 20.0,
  'income': 98.20000000000002,
  'roi': 3.910000000000001},
 'facebook': {'cost': 20.0,
  'income': 90.60000000000001,
  'roi': 3.5300000000000002},
 'direct': {'cost': 20.0, 'income': 88.4, 'roi': 3.4200000000000004},
 'vk': {'cost': 8.0, 'income': 29.0, 'roi': 2.625}}